In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# chat = ChatOpenAI(temperature=0.1)
chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [StreamingStdOutCallbackHandler]
)






## predict message 

메세지 component import

AIMessage: ai가 보내는 메세지<br>
SystemMessage : 우리가 llm에 설정을 제공하기 위한 message  
HumanMessage: 사람이 보낸 메세지



In [ ]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage

messgage = [
    SystemMessage(
        content = "you are a geograghy expert. And you only reply in Italian"
    ),
    AIMessage(
        content="ciao, mi chiamo paolo"
    ),
    HumanMessage(
        content= "What is the distance between Mexico and Thailand. Also what is your name?"
    )
]

chat.predict_messages(messages)

## prompt templates

In [6]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "what is the distance between {contry_a} and {contry_b}. also, what is your name?"),
])

prompt = template.format_messages(language= "korea",name = "박준수",contry_a="mexico", contry_b ="thailand")

chat.predict_messages(prompt)


AIMessage(content='안녕하세요! 저는 박준수입니다. 멕시코와 태국 사이의 거리는 대략적으로 17,000 킬로미터입니다.')

## outputparser and lcel  


In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("hello,how, are, you ")

template = ChatPromptTemplate.from_messages(
    [("system", "you are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do not reply with anything else."),
     ("human", "{question}"),
     
     ]
)

['hello', 'how', ' are', ' you']

In [ ]:

# prompt = template.format_messages(
#     max_items = 10 ,
#     question = "what are the planets"
# )

# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

## LangChain Expression Language (LCEL)


In [ ]:
chain =  template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question":"what are the poketmons?"
})

In [11]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. you create easy to follow recipies for any type of cuising with easy to find ingredient."),
    ("human", "i want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [12]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a vegetarian chef specialized on making traditional recipies vegetarian. you find alternative ingredients and explain their preparation. you don't radically modify the recipe. if there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "indian"
})
# veg_chain.invoke({
#     "recipt": "chatmodel"
# })